# Detekcia bleskov v rádiových spektrogramoch založená na konvolučných neurónových sieťach

## Príprava prostredia

Príkazy na inštaláciu potrebných knižníc.

In [ ]:
!pip install PyPDF2
!pip install pdf2image
!pip install os-sys
!pip install temp
!conda install -c conda-forge poppler -y
!pip install zipfile38
!pip install pycopy-fnmatch
!pip install python-csv
!pip install pandas
!pip install numpy
!pip install jsonlib
!pip install python-math
!pip install opencv-python
!pip install Pillow

Import potrebných knižníc.

In [1]:
import PyPDF2
import os
import tempfile
import zipfile,fnmatch
import csv
import pandas as pd
import numpy as np
import json
import math 
import cv2
from PIL import Image, ImageDraw
import operator
from skimage.io import imread
from PyPDF2 import PdfFileWriter, PdfFileReader
from pdf2image import convert_from_path, convert_from_bytes
from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)
from PIL import Image

## Terminalový príkaz na stiahnutie obrázkov priamo zo servera UFA AV ČR

Je potrebné url modifikovať na základe potreby.

In [ ]:
wget -A zip -r -l 3 http://bleska.ufa.cas.cz/lsbb/storage/elm/2015

## Rozbalenie a premazanie priečinkov stiahnutých dát

### Vstupy

rootPath: cesta ku priečinku, ktorý obsahuje výstupy analyzátora stiahnuté zo servera UFA AV ČR

In [2]:
rootPath = r"files/zip_from_web_unpacked"

In [3]:
####### UNZIP
pattern = '*.zip'
for root, dirs, files in os.walk(rootPath):
    for filename in fnmatch.filter(files, pattern):
        zipfile.ZipFile(os.path.join(root, filename)).extractall(os.path.join(root, os.path.splitext(filename)[0]))
        os.remove(os.path.join(root, filename))

for root, dirs, files in os.walk(rootPath):
    for file in files:
        if file.endswith(".pdf")==False:
            os.remove(os.path.join(root, file))

## Vytvorenie sprektrogramov ako vstupu pre neurónovú sieť

### Vstupy
left: parameter vzdialenosti ľavej hrany spektrogramu od ľavej hrany obrázka, na ktorom sa spektrogram nachádza (v pixeloch)<br>
upper: parameter vzdialenosti hornej hrany spektrogramu od hornej hrany obrázka, na ktorom sa spektrogram nachádza (v pixeloch)<br>
right: parameter vzdialenosti pravej hrany spektrogramu od ľavej hrany obrázka, na ktorom sa spektrogram nachádza (v pixeloch)<br>
lower: parameter vzdialenosti spodnej hrany spektrogramu od vrchnej hrany obrázka, na ktorom sa spektrogram nachádza (v pixeloch)<br>
rootPath: cesta ku priečinku, ktorý obsahuje výstupy analyzátora stiahnuté rozbalené zo servera UFA AV ČR

In [4]:
left=154
upper=1392
right=1575
lower=1554

In [5]:
####### CROP
for root, dirs, files in os.walk(rootPath):
    pole=[]
    for file in files:
        if file.endswith(".pdf"):
            print("####################################")
            print("processing %s" %root)
            pole=[]
            pdfFileObj = open(os.path.join(root, file), 'rb')
            pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
            pageObjDate = pdfReader.getPage(0)
            textDate=pageObjDate.extractText()
            if "from " in textDate:
                datum=textDate[textDate.find('from ')+5:textDate.find('from ')+29]
            else: 
                print("pdf from %s is empty, lets continue")
                break
            datum=datum.replace("-", "")
            datum=datum.replace("T", "_")
            datum=datum.replace(".", "_", 1)
            datum=datum.replace(".", "")
            datum=datum.replace(":", "")
            for j in range(pdfReader.numPages):
                pageObj = pdfReader.getPage(j)
                text=pageObj.extractText()
                if "Part" in text:
                    pole.append(text[text.find('Part')+5:text.find('/')])
                else:
                    pole.append(0)

            with tempfile.TemporaryDirectory() as path:
                images_from_path = convert_from_path(os.path.join(root, file), output_folder=path)

            i=0
            for page in images_from_path:
                page.save(os.path.join(root,"%s. %s.jpg" % (i, pole[i])), 'JPEG')   
                i+=1

            os.remove(os.path.join(root, "0. 0.jpg"))
            os.remove(os.path.join(root, "%s. %s.jpg" % (i-1, pole[i-1])))
            pole.remove(pole[0])
            pole.remove(pole[i-2])
            for i in pole:
                im = Image.open(os.path.join(root, "%s. %s.jpg" % (str(pole.index(i)+1), i)))
                im.crop((left, upper, right, lower)).save(os.path.join(root, "ch2_%s_s%s.jpg"%(datum, i)))
                os.remove(os.path.join(root, "%s. %s.jpg" % (str(pole.index(i)+1), i)))
            os.remove(os.path.join(root, "data.pdf"))
        else:
            continue

####################################
processing files/zip_from_web_unpacked/elm_20150515_200446_pics
####################################
processing files/zip_from_web_unpacked/elm_20150515_220450_pics


## Spustenie automatickej detekcie bleskov

### Vstupy
detect.py: cesta k súboru detect.py<br>
source: parameter obsahujúci cestu k priečinku, v ktorom je dataset obrázkov a textových súborov v jednom<br>
weights: parameter obsahujúci cestu k váham, pomocou ktorých chceme detekciu vykonať<br>
iou: hodnota thresholdu IoU<br>
conf: hodnota confidence thresholdu<br>
img: veľkosť obrázka, na ktorom bude trénovanie prebiehať v pixeloch<br>
save-txt: parameter, na základe ktorého sa budú detaily detekcie ukladať aj do textových súborov<br>
save-conf: parameter, na základe ktorého sa budú detaily confidence (istoty modelu) ukladať aj do textových súborov

In [36]:
!python yolo/detect.py --source ./yolo/test_img --weights yolo/runs/train/exp17/weights/best.pt --iou 0.45 --conf 0.3 --img 1184 --save-txt --save-conf

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.3, device='', exist_ok=False, hide_conf=False, hide_labels=False, img_size=1184, iou_thres=0.45, line_thickness=3, name='exp', nosave=False, project='runs/detect', save_conf=True, save_crop=False, save_txt=True, source='./yolo/test_img', update=False, view_img=False, weights=['yolo/runs/train/exp17/weights/best.pt'])
requirements: /home/jovyan/data/lightning/adrian/pics/BP_Kundrat_Adrian/requirements.txt not found, check failed.
YOLOv5 🚀 v5.0-97-gffb47ff torch 1.7.0 CUDA:0 (Quadro RTX 4000, 7982.3125MB)

Fusing layers... 
Model Summary: 224 layers, 7056607 parameters, 0 gradients
image 1/218 /home/jovyan/data/lightning/adrian/pics/BP_Kundrat_Adrian/yolo/test_img/ch1_20150902_230932_991_s93.jpg: 160x1184 11 Blesks, 4 Blesk s chvostikoms, Done. (0.019s)
image 2/218 /home/jovyan/data/lightning/adrian/pics/BP_Kundrat_Adrian/yolo/test_img/ch1_20150902_231433_221_s109.jpg: 160x1184 17 Blesks, 2 Blesk s chvostikoms, Done.

## Spustenie vyhodnotenia metrík neurónovej siete

### Vstupy
test.py: cesta k súboru test.py<br>
weights: parameter obsahujúci cestu k váham, pomocou ktorých chceme test vykonať<br>
data: cesta k súboru yaml, ktorý obsahuje cesty ku train, text a val množinám<br>
img: veľkosť obrázka, na ktorom bude trénovanie prebiehať v pixeloch
batch-size: hodnota batch-size pre test<br>
iou: hodnota thresholdu IoU<br>
conf: hodnota confidence thresholdu<br>
task: množina, na ktorej sa bude test vykonávať (možno vybrať z train, test a val)

In [37]:
!python yolo/test.py --weights ./yolo/runs/train/exp17/weights/best.pt --data ./yolo/data/blesky.yaml --img 1184 --batch-size 4 --iou 0.45 --conf 0.3 --task 'test' 

Namespace(augment=False, batch_size=4, conf_thres=0.3, data='./yolo/data/blesky.yaml', device='', exist_ok=False, img_size=1184, iou_thres=0.45, name='exp', project='runs/test', save_conf=False, save_hybrid=False, save_json=False, save_txt=False, single_cls=False, task='test', verbose=False, weights=['./yolo/runs/train/exp17/weights/best.pt'])
requirements: /home/jovyan/data/lightning/adrian/pics/BP_Kundrat_Adrian/requirements.txt not found, check failed.
YOLOv5 🚀 v5.0-97-gffb47ff torch 1.7.0 CUDA:0 (Quadro RTX 4000, 7982.3125MB)

Fusing layers... 
Model Summary: 224 layers, 7056607 parameters, 0 gradients
test: Scanning 'yolo/data/Test_batch.cache' images and labels... 218 found, 0 mi
               Class      Images      Labels           P           R     mAP@.45
               Class      Images      Labels           P           R     mAP@.45
               Class      Images      Labels           P           R     mAP@.45
               Class      Images      Labels           P      

## Vytvorenie tabuľky z výstupov neurónovej siete

route: názov priečnika obsahujúceho detekciu<br>
imagePath: cesta k priečinku, v ktorom sa nachádza množina obrázkov použitá pri detekcii<br>
rootPath: cesta k priečinku obsahujúcemu textové súbory z detekcie<br>
path: cesta pre uloženie výstupného súboru

In [38]:
route="exp"
imagePath="yolo/test_img"
rootPath = r"runs/detect/" + route + "/labels"
path = "files"

In [39]:
imageNR=1

with open(os.path.join(path, "output" + route + ".csv"), "w") as outputCsv:    
    fieldnamesOutput = ['image','event', 'date', 'second', 'milisecond', 'tweek', 'f_min<2kHz', 'confidence']
    tableWriter = csv.DictWriter(outputCsv, fieldnames=fieldnamesOutput, delimiter=',')
    tableWriter.writeheader()
    
    for root, dirs, files in os.walk(rootPath):
        for file in files:
            with open(os.path.join(root, file), "r") as csv_file:
                reader = csv.reader(csv_file, delimiter=' ')
                sortedlist = sorted(reader, key=operator.itemgetter(1))
            with open(os.path.join(root, file), "w") as csv_file1:
                writer = csv.writer(csv_file1, delimiter=' ')
                writer.writerows(sortedlist)

            with open(os.path.join(root, file), "r") as coordFile:

                fieldnamesCoords = ['label','x', 'y', 'width', 'height', 'conf']
                eventNR=1
                coordReader = csv.DictReader(coordFile, fieldnames=fieldnamesCoords, delimiter=' ')
                try:
                    img = Image.open(os.path.join(imagePath, file.split(".")[0]+".jpg"))
                except:
                    break
                w, h = img.size
                widthms=w/1000

                for row in coordReader:
                    widthCoord=round(float(row['width'])*w)
                    xCoord=round((float(row['x'])*w)-(float(widthCoord)/2))
                    im = img.crop((xCoord, 0, xCoord+widthCoord, 162)).save(os.path.join(root, "1.jpg"))
                    red = imread(os.path.join(root, "1.jpg"))[:, :, 0]
                    os.remove(os.path.join(root, "1.jpg"))

                    red=np.delete(red,len(red)-1,0)
                    red=np.delete(red,len(red)-1,0)

                    transposered=np.transpose(red)

                    middle=0
                    indexOfMid=0
                    currentIndex=1
                    for i in transposered:
                        if sum(i) > middle:
                            middle=sum(i)
                            indexOfMid=currentIndex
                        currentIndex+=1

                    milsec=round((indexOfMid+xCoord)/widthms)

                    j=0
                    celyobr=0
                    konec=0
                    while(j<160):
                        celyobr+=sum(red[j])
                        if j>138:
                            konec+=sum(red[j])

                        j+=1

                    if (float((konec*100)/celyobr))>9.5:
                        vysledok=1
                    else: vysledok=0
                    tableWriter.writerow({
                        'image':imageNR, 
                        'event':eventNR, 
                        'date':file[4:19], 
                        'second':file[file.find("s")+1:file.find(".txt")], 
                        'milisecond':milsec, 
                        'tweek':row['label'], 
                        'f_min<2kHz':vysledok,
                        'confidence':round(float(row['conf']),3)})
                    eventNR+=1
                    # print(row)

                imageNR+=1

